# SyftBox SDK - Python Examples

This notebook demonstrates the Python bindings for syftbox-sdk.

## Setup

First, build and install the wheel:
```bash
cd syftbox-sdk/python
uv tool run maturin build --release
uv pip install target/wheels/syftbox_sdk-*.whl
```

In [ ]:
!uv pip list

In [ ]:
import syftbox_sdk
print(f"SyftBox SDK version: {syftbox_sdk.__version__}")

## 1. Identity Provisioning

Generate a crypto identity (keypair) for a user. This creates:
- Private key in `.syc/keys/`
- Public DID bundle in `datasites/<email>/public/crypto/did.json`

In [ ]:
from pathlib import Path
import shutil

# Use local directory for testing (avoids macOS /var vs /private/var symlink issues)
TEST_DIR = Path(__file__).parent.resolve() / "test_data" if "__file__" in dir() else Path.cwd() / "test_data"
TEST_DIR = TEST_DIR.resolve()

# Clean and recreate
if TEST_DIR.exists():
    shutil.rmtree(TEST_DIR)
TEST_DIR.mkdir(parents=True, exist_ok=True)

print(f"Test directory: {TEST_DIR}")

In [ ]:
# Provision identity for Alice
alice_result = syftbox_sdk.provision_identity(
    identity="alice@test.local",
    data_root=str(TEST_DIR / "alice"),
)

print(f"Identity: {alice_result.identity}")
print(f"Generated: {alice_result.generated}")
print(f"Vault path: {alice_result.vault_path}")
print(f"Bundle path: {alice_result.bundle_path}")
print(f"Public bundle: {alice_result.public_bundle_path}")
if alice_result.recovery_mnemonic:
    print(f"Recovery mnemonic: {alice_result.recovery_mnemonic}")

In [ ]:
# Provision identity for Bob
bob_result = syftbox_sdk.provision_identity(
    identity="bob@test.local",
    data_root=str(TEST_DIR / "bob"),
)

print(f"Identity: {bob_result.identity}")
print(f"Generated: {bob_result.generated}")

In [ ]:
# Check what was created
!ls -la {TEST_DIR}/alice/.syc/
print("---")
!ls -la {TEST_DIR}/alice/datasites/alice@test.local/public/crypto/

## 2. Bundle Exchange

Import each other's public bundles so they can encrypt messages for each other.

In [ ]:
# Import Bob's public bundle into Alice's vault
verified_bob = syftbox_sdk.import_bundle(
    bundle_path=bob_result.public_bundle_path,
    vault_path=alice_result.vault_path,
    expected_identity="bob@test.local",
)
print(f"Alice imported Bob's bundle: {verified_bob}")

# Import Alice's public bundle into Bob's vault
verified_alice = syftbox_sdk.import_bundle(
    bundle_path=alice_result.public_bundle_path,
    vault_path=bob_result.vault_path,
    expected_identity="alice@test.local",
)
print(f"Bob imported Alice's bundle: {verified_alice}")

In [ ]:
# Check bundles directory
!ls -la {TEST_DIR}/alice/.syc/bundles/

## 3. SyftBoxStorage - Basic Operations

Create storage instances for reading/writing files.

In [ ]:
# Create storage for Alice (with encryption)
alice_storage = syftbox_sdk.SyftBoxStorage(
    root=str(TEST_DIR / "alice" / "datasites"),
    vault_path=alice_result.vault_path,
    disable_crypto=False,
)

print(f"Alice storage uses crypto: {alice_storage.uses_crypto()}")

In [ ]:
# Create storage for Bob (with encryption)
bob_storage = syftbox_sdk.SyftBoxStorage(
    root=str(TEST_DIR / "bob" / "datasites"),
    vault_path=bob_result.vault_path,
    disable_crypto=False,
)

print(f"Bob storage uses crypto: {bob_storage.uses_crypto()}")

## 4. Encrypted Write with Shadow Pattern

Write encrypted data using the shadow folder pattern:
- Plaintext is cached locally in `unencrypted/`
- Encrypted version is written to `datasites/` for syncing

In [ ]:
# Create directories
alice_shared = TEST_DIR / "alice" / "datasites" / "alice@test.local" / "shared" / "messages"
alice_shared.mkdir(parents=True, exist_ok=True)

# Alice writes an encrypted message for Bob
message = b"Hello Bob! This is a secret message from Alice."
message_path = str(alice_shared / "secret.txt")

alice_storage.write_with_shadow(
    absolute_path=message_path,
    data=message,
    recipients=["bob@test.local"],  # Encrypt for Bob
    hint="test-message",
    overwrite=True,
)

print(f"Wrote encrypted message to: {message_path}")

In [ ]:
# Check the encrypted file (should have SYC1 header)
encrypted_data = Path(message_path).read_bytes()
print(f"File size: {len(encrypted_data)} bytes")
print(f"First 4 bytes (header): {encrypted_data[:4]}")
print(f"Is encrypted: {encrypted_data[:4] == b'SYC1'}")

In [ ]:
# Show hex dump of first 64 bytes
!xxd {message_path} | head -4

## 5. Encrypted Read with Shadow Pattern

Bob reads and decrypts the message Alice sent.

In [ ]:
# Simulate sync: copy Alice's encrypted file to Bob's view of Alice's datasite
bob_view_of_alice = TEST_DIR / "bob" / "datasites" / "alice@test.local" / "shared" / "messages"
bob_view_of_alice.mkdir(parents=True, exist_ok=True)

# Copy the encrypted file (simulating SyftBox sync)
shutil.copy(message_path, bob_view_of_alice / "secret.txt")
print(f"Synced file to Bob's view: {bob_view_of_alice / 'secret.txt'}")

In [ ]:
# Bob reads and decrypts the message
bob_message_path = str(bob_view_of_alice / "secret.txt")
decrypted = bob_storage.read_with_shadow(bob_message_path)

print(f"Decrypted message: {bytes(decrypted).decode('utf-8')}")

## 6. Plaintext Operations (No Encryption)

You can also use the storage without encryption for testing.

In [ ]:
# Create storage without encryption
plain_storage = syftbox_sdk.SyftBoxStorage(
    root=str(TEST_DIR / "plain"),
    disable_crypto=True,
)

print(f"Plain storage uses crypto: {plain_storage.uses_crypto()}")

In [ ]:
# Write plaintext
plain_dir = TEST_DIR / "plain" / "data"
plain_dir.mkdir(parents=True, exist_ok=True)

plain_storage.write_bytes(
    str(plain_dir / "hello.txt"),
    b"Hello World!",
    overwrite=True,
)

# Read it back
content = plain_storage.read_bytes(str(plain_dir / "hello.txt"))
print(f"Content: {bytes(content).decode('utf-8')}")

## 7. SyftBoxApp - Application Structure

Helper for creating standard SyftBox app directory structure.

In [ ]:
# Create app structure for Alice
app = syftbox_sdk.SyftBoxApp(
    data_dir=str(TEST_DIR / "alice"),
    email="alice@test.local",
    app_name="my_app",
)

print(f"App properties:")
print(f"  app_name: {app.app_name}")
print(f"  email: {app.email}")
print(f"  data_dir: {app.data_dir}")
print(f"  rpc_dir: {app.rpc_dir}")

# Register an RPC endpoint
endpoint_path = app.register_endpoint("compute")
print(f"\nRegistered endpoint 'compute' at: {endpoint_path}")
print(f"Endpoint exists: {app.endpoint_exists('compute')}")
print(f"All endpoints: {app.list_endpoints()}")
print(f"SyftURL: {app.build_syft_url('compute')}")

In [ ]:
# Check created directories
!find {TEST_DIR}/alice/datasites -type d | head -20

## Cleanup

In [ ]:
# Clean up test directory
shutil.rmtree(TEST_DIR)
print(f"Cleaned up: {TEST_DIR}")

## Summary

The syftbox-sdk Python bindings provide:

1. **`provision_identity(identity, data_root)`** - Generate keypair and public DID
2. **`import_bundle(bundle_path, vault_path, expected_identity)`** - Import peer's public key
3. **`SyftBoxStorage(root, vault_path, disable_crypto, debug)`** - Encrypted file I/O
   - `.write_with_shadow()` - Write encrypted with plaintext cache
   - `.read_with_shadow()` - Read and decrypt
   - `.write_bytes()` / `.read_bytes()` - Basic I/O
4. **`SyftBoxApp(data_dir, email, app_name)`** - App directory structure helper
   - `.app_name`, `.email`, `.data_dir`, `.rpc_dir` - Properties
   - `.register_endpoint()` - Create RPC endpoint
   - `.endpoint_exists()` / `.list_endpoints()` - Query endpoints
   - `.build_syft_url()` - Build syft:// URL for endpoint